In [8]:
# read list of nuclear charges, give energy predictions up to nth order

In [74]:
import numpy as np
import pandas as pd
import qml
import scipy.spatial as scs
import scipy.interpolate as sci

In [61]:
def get_predictions(comb = [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 7]):
    #rho, dsingle, dneigh1, dneigh2, dneigh3, dneigh4, dneigh5 = read_densities()
        
    E =0 #-758.072029908548 # base energy
    deltaZ = np.array(comb) - 6
        
    zs = np.array([int(_) for _ in comb])
    zsref = np.zeros(20) + 6
    
    E -= get_nucnuc(zsref)
    E += get_nucnuc(zs)
    
    # 0-th order, no rotation necessary, should be hard zero
    #ds = np.linalg.norm(grid_points - c.coordinates[0], axis=1)
    #es = (rho * grid_weights / ds).sum()
    #for idx, Z in enumerate(comb):
    #    if deltaZ[idx] == 0:
    #        continue
    #    E += np.sum(deltaZ[idx] * rho * grid_weights / ds)
    
    # 1st order
    changed_site, grid_points, grid_densweight = read_grid_first_order()
    for idx, Z in enumerate(comb):
        if deltaZ[idx] == 0:
            continue
        mapping = build_reindexing_1(idx, changed_site)
        
        for j in range(20): 
            if deltaZ[mapping[j]] == 0:
                continue
            ds = np.linalg.norm(grid_points - c.coordinates[j], axis=1)*1.8897259885789
            E += np.sum(deltaZ[idx] * deltaZ[mapping[j]] * grid_densweight / ds)/2
    
    # 2nd order
    for idx_i, Z_i in enumerate(comb):
        if deltaZ[idx_i] == 0:
            continue
        for idx_j, Z_j in enumerate(comb):
            if deltaZ[idx_j] == 0:
                continue
            
            # 
            
            t_i, t_j, grid_points, grid_densweight = get_deriv(idx_i, idx_j)
            
            if idx_i == idx_j:
                mapping = build_reindexing_1(idx_i, t_i)
            else:
                mapping = build_reindexing_2(idx_i, idx_j, t_i, t_j)
            for j in range(20): 
                if deltaZ[mapping[j]] == 0:
                    continue
                ds = np.linalg.norm(grid_points - c.coordinates[j], axis=1)
                E += np.sum(deltaZ[idx] * deltaZ[mapping[j]] * grid_densweight / ds)/6
    
    return E
    # 0-1 neigh1
    # 0-2 neigh2
    # 0-8 neigh3
    # 0-10 neigh4
    # 0-16 neigh5
get_predictions([int(_) for _ in '57766576666555776675'])

/home/guido/miniconda3/envs/analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


-2.5593708955421985

In [4]:
c = qml.Compound('../../test/c20.xyz')

In [73]:
def _get_nn(refsite):
    distances = np.linalg.norm(c.coordinates - c.coordinates[refsite], axis=1)
    return np.argsort(distances)[1:4]
def get_rotation_1(refsite, ontosite):
    cog = np.mean(c.coordinates, axis=0)
    valid = False
    for Ann in _get_nn(refsite):
        for Bnn in _get_nn(ontosite):
            A = c.coordinates[[refsite, Ann]]
            B = c.coordinates[[ontosite, Bnn]]
            rot = scs.transform.Rotation.match_vectors(A, B)[0]
            transformed = rot.apply(c.coordinates)
            found = []
            for site in range(len(c.coordinates)):
                ds = np.linalg.norm(transformed - c.coordinates[site], axis=1)
                if min(ds) < 1e-5:
                    found.append(np.argmin(ds))
            if set(found) == set([_ for _ in range(len(c.coordinates))]) and found[refsite] == ontosite:
                valid = True
                break
        if valid:
            break
    if not valid:
        raise ValueError('no solution')
    return rot
def build_reindexing_1(refsite, ontosite):
    cog = np.mean(c.coordinates, axis=0)
    valid = False
    for Ann in _get_nn(refsite):
        for Bnn in _get_nn(ontosite):
            A = c.coordinates[[refsite, Ann]]
            B = c.coordinates[[ontosite, Bnn]]
            rot = scs.transform.Rotation.match_vectors(A, B)[0]
            transformed = rot.apply(c.coordinates)
            found = []
            for site in range(len(c.coordinates)):
                ds = np.linalg.norm(transformed - c.coordinates[site], axis=1)
                if min(ds) < 1e-5:
                    found.append(np.argmin(ds))
            if set(found) == set([_ for _ in range(len(c.coordinates))]) and found[refsite] == ontosite:
                valid = True
                break
        if valid:
            break
    if not valid:
        raise ValueError('no solution')
    return found

build_reindexing_1(5, 0)

[4, 3, 11, 12, 13, 0, 1, 2, 9, 10, 16, 17, 18, 14, 5, 8, 15, 19, 6, 7]

In [91]:
def build_reindexing_2(refsite1, refsite2, ontosite1, ontosite2):
    cog = np.mean(c.coordinates, axis=0)
    valid = False
    A = c.coordinates[[refsite1, refsite2]]
    B = c.coordinates[[ontosite1, ontosite2]]
    rot = scs.transform.Rotation.match_vectors(A, B)[0]
    transformed = rot.apply(c.coordinates)
    found = []
    for site in range(len(c.coordinates)):
        ds = np.linalg.norm(transformed - c.coordinates[site], axis=1)
        if min(ds) < 1e-5:
            found.append(np.argmin(ds))
    if set(found) == set([_ for _ in range(len(c.coordinates))]) and found[refsite1] == ontosite1 and found[refsite2] == ontosite2:
        pass
    else:
        raise ValueError('no solution')
    return found
build_reindexing_2(0, 5, 7, 1)

[7, 6, 19, 15, 8, 1, 0, 5, 14, 18, 17, 16, 10, 9, 2, 13, 12, 11, 3, 4]

In [28]:
def read_DENSITY(fn):
    with open(fn, 'r') as fh:
        _ = np.fromfile(fh, 'i4')
        q = _[3:-1].view(np.float64)
        ccdensity = q.reshape((-1, 10))
    ccdensity = ccdensity[:, 1:6]
    return ccdensity[:, :3], ccdensity[:, 3], ccdensity[:, 4]

def read_grid_first_order():
    changed_site = 0
    delta = 0.1
    
    upgrid, upweight, updens = read_DENSITY('c20-data/derivatives/order-1/site-0-up/DENSITY')
    dngrid, dnweight, dndens = read_DENSITY('c20-data/derivatives/order-1/site-0-dn/DENSITY')
    
    if not np.allclose(upgrid, dngrid):
        raise ValueError('Grid?')
        
    if not np.allclose(upweight, dnweight):
        raise ValueError('Grid?')
    
    return changed_site, upgrid, ((updens - dndens) / delta)*upweight

(0, array([[-14.45979167,  -7.67919738,  -4.7844303 ],
        [-14.12594505,  -6.47848004,  -4.16214482],
        [-13.98485046,  -7.04498829,  -6.58628751],
        ...,
        [ 14.12594505,   6.47848004,   4.16214293],
        [ 14.45978978,   7.67920116,   4.78443219],
        [ 15.35325833,   6.6146531 ,   3.73665988]]), array([7.80818106e-25, 6.67525811e-23, 2.61010655e-25, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]))

In [58]:
def get_nucnuc(zs):
    ds = scs.distance.squareform(scs.distance.pdist(c.coordinates))*1.8897259885789
    q = np.outer(zs, zs)/ds
    np.fill_diagonal(q, 0)
    return q.sum()/2    

In [79]:
def rotate_density(origgrid, origdens, target):
    rot = get_rotation_1(target, 0)
    newgrid = rot.apply(origgrid)
    transform = sci.Rbf(newgrid[:, 0], newgrid[:, 1], newgrid[:, 2], origdens, function='thin_plate')
    return transform(origgrid[:, 0], origgrid[:, 1], origgrid[:, 2])
    
def get_deriv(i, j):
    d = np.linalg.norm(c.coordinates[i] - c.coordinates[j])
    geo = np.argmin(np.abs(np.array(sorted(set(np.round(scs.distance.pdist(c.coordinates), 2)))) - d))
    sites = (0, (1, 2, 8, 10, 16)[geo])
    i, j = sites
    delta = 0.05
    
    assert i == 0
    midgrid, midweight, middens = read_DENSITY('c20-data/derivatives/order-0/site-all-cc/DENSITY')
    
    iupgrid, iupweight, iupdens = read_DENSITY('c20-data/derivatives/order-1/site-0-up/DENSITY')
    idngrid, idnweight, idndens = read_DENSITY('c20-data/derivatives/order-1/site-0-dn/DENSITY')
    
    if i == j:
        deriv = (iupdens + idndens - 2 * middens)/(delta**2)
    else:
        rhojup = rotate_density(iupgrid, iupdens, j)
        rhojdn = rotate_density(idngrid, idndens, j)
        upgrid, upweight, updens = read_DENSITY('c20-data/derivatives/order-2/site-0-%d-up/DENSITY' % j)
        dngrid, dnweight, dndens = read_DENSITY('c20-data/derivatives/order-2/site-0-%d-dn/DENSITY' % j)
        
        deriv = (updens + dndens + 2 * middens - iupdens - idndens - rhojup - rhojdn) / (2 * delta**2)
    
    return i, j, midgrid, midweight, deriv
get_deriv(0, 1)

MemoryError: 

1